In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np

import sys
from pathlib import Path

# Add project paths
project_root = Path.cwd().parent

# Import experiment management from HDMF src folder
from fastHDMF.simulation_runner import HDMFSimulationRunner
from fastHDMF.experiment_manager import ExperimentManager


def compute_fcd(data, wsize, overlap, isubdiag):
    T, N = data.shape
    win_start = np.arange(0, T - wsize - 1, wsize - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    #print(fcd.shape)
    #print(data.shape)
    #print((data[win_start[2]:win_start[2] + wsize + 1, :]).shape)
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + wsize + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0],isubdiag[1]]
    return fcd




In [ ]:
exp = ExperimentManager(project_root=project_root, config_path="Default", results_dir="/network/iss/home/ivan.mindlin/Repos/fastHDMF/results")
runner = HDMFSimulationRunner(experiment_manager=exp)
# Extract the first matrix from the sc_matrices dictionary in the experiment manager
sc_matrix = list(exp.sc_matrices.values())[0]
task = runner.tasks[0] # The runner creates the tasks that will be used to create the params for the simulation
# Add the sc_matrix to the task dictionary
task['sc_matrix'] = sc_matrix

sim = runner.run_one_simulation(task)

2025-09-25 12:06:38,722 - hdmf_experiment_Default - INFO - Experiment 'Default' initialized
2025-09-25 12:06:38,723 - hdmf_experiment_Default - INFO - Config: State-of-the-art DMF simulation with default parameters
2025-09-25 12:06:38,724 - hdmf_experiment_Default - INFO - No save flags set; results will not be written to disk.


INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/Default.yaml
Creating IPP list from SC matrices in: AAL
Found 1 IPPs from SC matrices


In [6]:
len(runner.tasks)

1

## Get empirical data

In [7]:

from scipy.io import loadmat
emp_data = loadmat('../data/ts_coma24_AAL_symm_withSC.mat')
emp_data = np.squeeze(np.array(emp_data['timeseries_CNT24_symm']))
emp_data = np.stack(emp_data.flatten())[:,:,exp.current_config['simulation']['burnout']:]
isubfcd = np.triu_indices(params['N'], 1)
params["wsize"] = 30
params["overlap"] = 29

emp_fcds = []
for sub in range(emp_data.shape[0]):
    fcds = compute_fcd(emp_data[sub, :, :].T, params["wsize"], params['overlap'], isubfcd)
    upp_tr_fcd = np.corrcoef(fcds.T)
    emp_fcds.append(upp_tr_fcd)

emp_fc = np.mean(np.array([np.corrcoef(emp_data[sub, :, :]) for sub in range(emp_data.shape[0])]), axis=0)

# remove the diagonal

emp_fc = emp_fc - np.eye(emp_fc.shape[0])

emp_fcd = np.hstack((emp_fcds))

In [8]:
res

{'metadata': {'experiment_id': 'Fitting_Grid',
  'original_config_path': '/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Fitting_Grid.yaml',
  'start_time': '2025-09-22T13:16:20.160444',
  'config': {'simulation': {'nb_steps': 500000,
    'obj_rate': 3.44,
    'with_decay': True,
    'with_plasticity': True,
    'TR': 2.4,
    'flp': 0.01,
    'fhp': 0.1,
    'burnout': 8,
    'over': {'seed': {'fun': 'np.arange', 'args': [1, 30, 1]}},
    'parallel': True},
   'grid': {'lrj': {'fun': 'np.logspace', 'args': [0, 5, 2]},
    'G': {'fun': 'np.arange', 'args': [0.1, 8.1, 2]}},
   'data': {'test_mode': False, 'max_subjects_test': 1, 'sc_root': 'AAL'},
   'output': {'save_full_outputs': True,
    'observables': [{'name': 'fc', 'signal': 'bold'},
     {'name': 'fcd', 'signal': 'bold'}]},
   'experiment': {'description': 'FC and FCD grid ', 'version': '1.0'}},
  'status': 'initialized',
  'observables_spec': [{'name': 'fc',
    'signal': ['bold'],
    'params': {'zero_diag':